In [2]:
from __future__ import annotations

from prompts import PROBLEM_PROMPT
from orchestrator import Orchestrator
from dev import DevAgent
from summary_agent import SummaryAgent


def run_iterations(n: int = 20) -> str:
    blackboard = ""
    agent = DevAgent()
    summarizer = SummaryAgent()

    for iteration in range(n):
        # 1) Orchestrate (pass total budget so it can plan)
        orch = Orchestrator(
            problem_description=PROBLEM_PROMPT,
            blackboard=blackboard,
            iteration=iteration,
            max_iterations=n,  # NEW: total task budget
        )
        step = orch.orchestrator_step()  # expected: {"text": "...", ...}
        instr = step["text"] if isinstance(step, dict) else step

        # 2) Dev agent executes
        report = agent.run(instr, iteration=iteration, max_repairs=3)

        # 3) Summarize and update blackboard
        out = summarizer.run(
            iteration=iteration,
            orchestrator_text=instr,
            current_blackboard=blackboard,
            dev_report=report,
            save_path=None,
        )

        print(f"\n=== Summary: Iteration {iteration} ===\n")
        print(out["summary_block"])
        blackboard = out["updated_blackboard"]

    return blackboard


if __name__ == "__main__":
    final_blackboard = run_iterations(5)
    print("\n=== Final Blackboard ===\n")
    print(final_blackboard)



=== Summary: Iteration 0 ===

Iteration 0
Asked: Train and evaluate two independent LightGBM regressors using 5-fold cross-validation, ensuring predictions are transformed back using expm1 and clipped to be non-negative. Save the submission file and metrics.
Did:
*   Trained two LightGBM regressors on log1p transformed targets using 5-fold CV.
*   Generated predictions, inverse transformed them using expm1, and clipped to >= 0.
*   Saved submission file `./0/submission.csv` and metrics file `./0/metrics.json`.
Result: Success. CV RMSLE: formation\_energy\_ev\_natom=0.0328, bandgap\_energy\_ev=0.0912, mean=0.0620. Train shape: (2400, 14), Test shape: (600, 12).

=== Summary: Iteration 1 ===

Iteration 1
Asked: Train and evaluate two independent XGBoost regressors using 5-fold cross-validation. Ensure predictions are transformed back using expm1 and clipped to be non-negative. Save the submission file and metrics.
Did:
*   Trained two XGBoost regressors on log1p transformed targets usin

In [5]:
import json
try:
    with open(f'.agent_workspace/{4}/attempt_0_report.json', 'r') as file:
        metrics = json.load(file)
except:
    pass
print(metrics['instructions'])

Usable local files:
- ./train.csv
- ./test.csv

Dev Context:
You are the Dev Agent.

Allowed files:
- ./train.csv, ./test.csv

Feature columns in test.csv:
id, spacegroup, number_of_total_atoms, percent_atom_al, percent_atom_ga, percent_atom_in,
lattice_vector_1_ang, lattice_vector_2_ang, lattice_vector_3_ang,
lattice_angle_alpha_degree, lattice_angle_beta_degree, lattice_angle_gamma_degree

Objective:
Predict for each material id in test.csv:
- formation_energy_ev_natom
- bandgap_energy_ev

Evaluation:
Column-wise RMSLE with log1p:
RMSLE = sqrt( (1/n) * Σ (log(1+p_i) - log(1+a_i))^2 ); final score = mean over the two targets.

Environment:
- Python 3.10+, CPU-only, deterministic; no internet or package installs.
- Available libraries: numpy, pandas, scikit-learn, lightgbm, xgboost, statsmodels, scipy.
- Return ONLY a single Python fenced block with self-contained code.

IO contract (always do):
1) Write ./4/submission.csv with EXACT header:
   id,formation_energy_ev_natom,bandgap_ener

In [6]:
import json
try:
    with open(f'.agent_workspace/{3}/attempt_0_report.json', 'r') as file:
        metrics = json.load(file)
except:
    pass
print(metrics['instructions'])

Usable local files:
- ./train.csv
- ./test.csv

Dev Context:
You are the Dev Agent.

Allowed files:
- ./train.csv, ./test.csv

Feature columns in test.csv:
id, spacegroup, number_of_total_atoms, percent_atom_al, percent_atom_ga, percent_atom_in,
lattice_vector_1_ang, lattice_vector_2_ang, lattice_vector_3_ang,
lattice_angle_alpha_degree, lattice_angle_beta_degree, lattice_angle_gamma_degree

Objective:
Predict for each material id in test.csv:
- formation_energy_ev_natom
- bandgap_energy_ev

Evaluation:
Column-wise RMSLE with log1p:
RMSLE = sqrt( (1/n) * Σ (log(1+p_i) - log(1+a_i))^2 ); final score = mean over the two targets.

Environment:
- Python 3.10+, CPU-only, deterministic; no internet or package installs.
- Available libraries: numpy, pandas, scikit-learn, lightgbm, xgboost, statsmodels, scipy.
- Return ONLY a single Python fenced block with self-contained code.

IO contract (always do):
1) Write ./submission.csv with EXACT header:
   id,formation_energy_ev_natom,bandgap_energy